In [1]:
import yfinance as yf
from datetime import datetime, timedelta
import pandas_ta as ta
import time

In [2]:
asset_list = [

    {'ticker':'AMZN', 'yfticker':yf.Ticker('AMZN'),'holding':False},
    {'ticker':'GOOG', 'yfticker':yf.Ticker('GOOG'),'holding':False},
    {'ticker':'NFLX', 'yfticker':yf.Ticker('NFLX'),'holding':False},
    {'ticker':'MSFT', 'yfticker':yf.Ticker('MSFT'),'holding':False},
    {'ticker':'TSLA', 'yfticker':yf.Ticker('TSLA'),'holding':False},
    {'ticker':'BTC-USD', 'yfticker':yf.Ticker('BTC-USD'),'holding':False},
    {'ticker':'ETH-USD', 'yfticker':yf.Ticker('ETH-USD'),'holding':False},

]

period_fast = 10
period_slow = 30
currently_holding = False
tradelog = []

In [3]:
import math
def get_pause():
    now = datetime.now()
    next_min = now.replace(second=0, microsecond=0) + timedelta(minutes=1)
    pause = math.ceil((next_min - now).seconds)
    print(f"Sleep for {pause}")
    return pause

In [ ]:
while True:
    for asset in asset_list:
        start_date = (datetime.now() - timedelta(days = 2)).strftime('%Y-%m-%d')
        df = asset['yfticker'].history(start=start_date, interval='1m')

        del df['Dividends']
        del df['Stock Splits']
        del df['Volume']

        df['SMA_fast'] = ta.sma(df['Close'], period_fast)
        df['SMA_slow'] = ta.sma(df['Close'], period_slow)

        price = df.iloc[-1]['Close']

        if df.iloc[-1]['SMA_fast'] > df.iloc[-1]['SMA_slow'] and not currently_holding:
            print(f"Buy {asset['ticker']}@{price}")
            tradelog.append({'date': datetime.now(), 'ticker': asset['ticker'], 'side': 'buy', 'price': price})
            currently_holding = True

        elif df.iloc[-1]['SMA_fast'] < df.iloc[-1]['SMA_slow'] and currently_holding:
            print(f"Sell {asset['ticker']}@{price}")
            tradelog.append({'date': datetime.now(), 'ticker': asset['ticker'], 'side': 'sell', 'price': price})
            currently_holding = False

    time.sleep(get_pause())